# 7 — Stacking Ensemble: Solar Generation & Electricity Price
Per-horizon stacking with 5-fold temporal CV. 4 tabular base models + TSO + PatchTST predictions → Ridge meta-learner.

**Requires**: Run notebook 06 first to generate PatchTST prediction CSVs.

In [1]:
import pandas as pd
import numpy as np
import json
import os
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
try:
    import lightgbm as lgb
except ImportError:
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'lightgbm'])
    import lightgbm as lgb

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

df = pd.read_parquet('../cleaned_data.parquet')
df['time'] = pd.to_datetime(df['time'], utc=True)

train_mask = df['time'].dt.year <= 2017
train_end = int(train_mask.sum())
test_start = train_end
context_length = 168
prediction_length = 24

print(f'Shape: {df.shape}')
print(f'Train: {train_end}, Test: {len(df) - train_end}')

Shape: (35056, 80)
Train: 26280, Test: 8776


In [2]:
def extract_tabular_features(df, target_col, weather_cols, window_start, h):
    """Extract features for one (window, horizon) pair."""
    target_idx = window_start + h

    # Weather at target hour
    weather = df[weather_cols].iloc[target_idx].values.astype(float)

    # Time features
    t = df['time'].iloc[target_idx]
    time_feats = np.array([t.hour, t.month, t.dayofweek, int(t.dayofweek >= 5)], dtype=float)

    # Context statistics on target series
    start_168 = max(0, target_idx - 168)
    start_24 = max(0, target_idx - 24)
    ctx_168 = df[target_col].iloc[start_168 : target_idx].values.astype(float)
    ctx_24 = df[target_col].iloc[start_24 : target_idx].values.astype(float)

    if len(ctx_24) == 0:
        ctx_24 = np.array([0.0])
    if len(ctx_168) == 0:
        ctx_168 = np.array([0.0])

    stats = np.array([
        np.mean(ctx_24), np.std(ctx_24), np.min(ctx_24), np.max(ctx_24),
        np.mean(ctx_168), np.std(ctx_168), np.min(ctx_168), np.max(ctx_168),
        ctx_168[-1],
    ])

    return np.concatenate([weather, time_feats, stats])

print('Feature extraction function defined')

Feature extraction function defined


In [3]:
# Load PatchTST predictions
pt_solar_train = pd.read_csv('patchtst_solar_train_predictions.csv')
pt_solar_test = pd.read_csv('patchtst_solar_predictions.csv')
pt_price_train = pd.read_csv('patchtst_price_train_predictions.csv')
pt_price_test = pd.read_csv('patchtst_price_predictions.csv')

print(f'PatchTST solar — train: {len(pt_solar_train)}, test: {len(pt_solar_test)}')
print(f'PatchTST price — train: {len(pt_price_train)}, test: {len(pt_price_test)}')

# Compute window starts (same as PatchTST notebook)
train_window_starts = list(range(context_length, train_end - prediction_length, prediction_length))
test_window_starts = []
for i in range(test_start, len(df) - prediction_length, prediction_length):
    if i - context_length >= 0:
        test_window_starts.append(i)

print(f'Train windows: {len(train_window_starts)}, Test windows: {len(test_window_starts)}')

PatchTST solar — train: 26112, test: 8736
PatchTST price — train: 26112, test: 8736
Train windows: 1088, Test windows: 364


In [4]:
def get_base_models():
    return {
        'xgb': xgb.XGBRegressor(n_estimators=200, max_depth=4, learning_rate=0.1,
                                random_state=42, verbosity=0, tree_method='hist'),
        'lgbm': lgb.LGBMRegressor(n_estimators=200, max_depth=4, learning_rate=0.1,
                                  random_state=42, verbose=-1),
        'ridge': Ridge(alpha=1.0),
        'rf': RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42, n_jobs=-1),
    }


def run_stacking(df, target_col, tso_col, weather_cols,
                 train_ws, test_ws, pt_train_df, pt_test_df,
                 prediction_length=24, n_folds=5, clip_min=None):
    """Per-horizon stacking with K-fold CV."""
    n_train = len(train_ws)
    n_test = len(test_ws)
    all_preds = np.zeros((n_test, prediction_length))
    all_actuals = np.zeros((n_test, prediction_length))
    all_tso = np.zeros((n_test, prediction_length))

    # Index PatchTST predictions by (window, horizon)
    pt_train_idx = pt_train_df.set_index(['window', 'horizon'])['patchtst_pred']
    pt_test_idx = pt_test_df.set_index(['window', 'horizon'])['patchtst_pred']

    model_names = list(get_base_models().keys())

    for h in range(prediction_length):
        # Extract features
        X_train = np.array([extract_tabular_features(df, target_col, weather_cols, ws, h)
                           for ws in train_ws])
        y_train = np.array([df[target_col].iloc[ws + h] for ws in train_ws])
        tso_train = np.array([df[tso_col].iloc[ws + h] for ws in train_ws])
        pt_train = np.array([pt_train_idx.loc[(w, h)] for w in range(n_train)])

        X_test = np.array([extract_tabular_features(df, target_col, weather_cols, ws, h)
                          for ws in test_ws])
        y_test = np.array([df[target_col].iloc[ws + h] for ws in test_ws])
        tso_test = np.array([df[tso_col].iloc[ws + h] for ws in test_ws])
        pt_test = np.array([pt_test_idx.loc[(w, h)] for w in range(n_test)])

        # 5-fold temporal CV for OOF predictions
        fold_size = n_train // n_folds
        oof_preds = {name: np.zeros(n_train) for name in model_names}

        for fold in range(n_folds):
            val_start = fold * fold_size
            val_end = (fold + 1) * fold_size if fold < n_folds - 1 else n_train
            train_idx = list(range(val_start)) + list(range(val_end, n_train))
            val_idx = list(range(val_start, val_end))

            for name in model_names:
                m = get_base_models()[name]
                m.fit(X_train[train_idx], y_train[train_idx])
                oof_preds[name][val_idx] = m.predict(X_train[val_idx])

        # Stack: OOF + TSO + PatchTST -> Ridge meta-learner
        meta_train = np.column_stack([oof_preds[n] for n in model_names] + [tso_train, pt_train])
        meta = Ridge(alpha=1.0)
        meta.fit(meta_train, y_train)

        # Retrain base models on full train -> predict test
        test_base_preds = {}
        for name in model_names:
            m = get_base_models()[name]
            m.fit(X_train, y_train)
            test_base_preds[name] = m.predict(X_test)

        meta_test = np.column_stack([test_base_preds[n] for n in model_names] + [tso_test, pt_test])
        final_pred = meta.predict(meta_test)

        if clip_min is not None:
            final_pred = np.clip(final_pred, clip_min, None)

        all_preds[:, h] = final_pred
        all_actuals[:, h] = y_test
        all_tso[:, h] = tso_test

        if (h + 1) % 6 == 0:
            print(f'  Horizon {h+1}/{prediction_length} done')

    return all_preds, all_actuals, all_tso

print('Stacking function defined')

Stacking function defined


In [5]:
# Solar stacking
solar_weather_cols = [
    'clouds_all_madrid', 'clouds_all_bilbao', 'clouds_all_barcelona',
    'clouds_all_seville', 'clouds_all_valencia',
    'temp_madrid', 'temp_bilbao', 'temp_barcelona',
    'temp_seville', 'temp_valencia',
    'temp_max_madrid', 'temp_max_bilbao', 'temp_max_barcelona',
    'temp_max_seville', 'temp_max_valencia',
    'humidity_madrid', 'humidity_bilbao', 'humidity_barcelona',
    'humidity_seville', 'humidity_valencia',
]

print('=== Stacking Ensemble for Solar ===')
solar_preds, solar_actuals, solar_tso = run_stacking(
    df, 'generation solar', 'forecast solar day ahead', solar_weather_cols,
    train_window_starts, test_window_starts,
    pt_solar_train, pt_solar_test,
    clip_min=0,
)
print('Solar stacking complete')

=== Stacking Ensemble for Solar ===
  Horizon 6/24 done
  Horizon 12/24 done
  Horizon 18/24 done
  Horizon 24/24 done
Solar stacking complete


In [6]:
# Solar metrics
flat_pred_s = solar_preds.flatten()
flat_act_s = solar_actuals.flatten()
flat_tso_s = solar_tso.flatten()

stack_mae_s = np.mean(np.abs(flat_act_s - flat_pred_s))
stack_rmse_s = np.sqrt(np.mean((flat_act_s - flat_pred_s) ** 2))
stack_mape_s = np.mean(np.abs((flat_act_s - flat_pred_s) / np.clip(np.abs(flat_act_s), 1, None))) * 100

tso_mae_s = np.mean(np.abs(flat_act_s - flat_tso_s))
tso_rmse_s = np.sqrt(np.mean((flat_act_s - flat_tso_s) ** 2))

print(f'{"Metric":<14} {"Stacking":>12} {"TSO":>12} {"Improvement":>12}')
print('-' * 52)
print(f'{"MAE (MW)":<14} {stack_mae_s:>12.1f} {tso_mae_s:>12.1f} {(1 - stack_mae_s / tso_mae_s) * 100:>+11.1f}%')
print(f'{"RMSE (MW)":<14} {stack_rmse_s:>12.1f} {tso_rmse_s:>12.1f} {(1 - stack_rmse_s / tso_rmse_s) * 100:>+11.1f}%')
print(f'{"MAPE (%)":<14} {stack_mape_s:>12.1f}')

Metric            Stacking          TSO  Improvement
----------------------------------------------------
MAE (MW)           203.7        141.1       -44.4%
RMSE (MW)          302.0        226.3       -33.5%
MAPE (%)           203.7


In [7]:
# Export stacking_solar.json
os.makedirs('../dashboard/public/data', exist_ok=True)

sample_data_s = []
for w in range(9, 16):
    if w >= len(test_window_starts):
        break
    ws = test_window_starts[w]
    for h in range(prediction_length):
        t = df['time'].iloc[ws + h]
        sample_data_s.append({
            'time': t.strftime('%Y-%m-%d %H:%M'),
            'actual': round(float(solar_actuals[w, h]), 1),
            'predicted': round(float(solar_preds[w, h]), 1),
            'tso': round(float(solar_tso[w, h]), 1),
        })

output_s = {
    'target': 'solar',
    'model': 'Stacking Ensemble (XGB + LGBM + Ridge + RF)',
    'prediction_length_hours': prediction_length,
    'context_length_hours': context_length,
    'metrics': {
        'mae': round(float(stack_mae_s), 1),
        'rmse': round(float(stack_rmse_s), 1),
        'mape': round(float(stack_mape_s), 1),
        'tso_mae': round(float(tso_mae_s), 1),
        'tso_rmse': round(float(tso_rmse_s), 1),
    },
    'sample_forecast': sample_data_s,
}

with open('../dashboard/public/data/stacking_solar.json', 'w') as f:
    json.dump(output_s, f, indent=2)

print(f'Saved stacking_solar.json (MAE: {output_s["metrics"]["mae"]} MW)')

Saved stacking_solar.json (MAE: 203.7 MW)


In [8]:
# Price stacking
price_weather_cols = [
    'pressure_madrid', 'pressure_bilbao', 'pressure_barcelona',
    'pressure_seville', 'pressure_valencia',
    'temp_madrid', 'temp_bilbao', 'temp_barcelona',
    'temp_seville', 'temp_valencia',
    'temp_max_madrid', 'temp_max_bilbao', 'temp_max_barcelona',
    'temp_max_seville', 'temp_max_valencia',
    'temp_min_madrid', 'temp_min_bilbao', 'temp_min_barcelona',
    'temp_min_seville', 'temp_min_valencia',
    'humidity_madrid', 'humidity_bilbao', 'humidity_barcelona',
    'humidity_seville', 'humidity_valencia',
    'wind_speed_madrid', 'wind_speed_bilbao', 'wind_speed_barcelona',
    'wind_speed_seville', 'wind_speed_valencia',
]

print('=== Stacking Ensemble for Price ===')
price_preds, price_actuals, price_tso = run_stacking(
    df, 'price actual', 'price day ahead', price_weather_cols,
    train_window_starts, test_window_starts,
    pt_price_train, pt_price_test,
)
print('Price stacking complete')

=== Stacking Ensemble for Price ===
  Horizon 6/24 done
  Horizon 12/24 done
  Horizon 18/24 done
  Horizon 24/24 done
Price stacking complete


In [9]:
# Price metrics
flat_pred_p = price_preds.flatten()
flat_act_p = price_actuals.flatten()
flat_tso_p = price_tso.flatten()

stack_mae_p = np.mean(np.abs(flat_act_p - flat_pred_p))
stack_rmse_p = np.sqrt(np.mean((flat_act_p - flat_pred_p) ** 2))
stack_mape_p = np.mean(np.abs((flat_act_p - flat_pred_p) / np.clip(np.abs(flat_act_p), 1, None))) * 100

tso_mae_p = np.mean(np.abs(flat_act_p - flat_tso_p))
tso_rmse_p = np.sqrt(np.mean((flat_act_p - flat_tso_p) ** 2))

print(f'{"Metric":<14} {"Stacking":>12} {"TSO":>12} {"Improvement":>12}')
print('-' * 52)
print(f'{"MAE (EUR/MWh)":<14} {stack_mae_p:>12.2f} {tso_mae_p:>12.2f} {(1 - stack_mae_p / tso_mae_p) * 100:>+11.1f}%')
print(f'{"RMSE (EUR/MWh)":<14} {stack_rmse_p:>12.2f} {tso_rmse_p:>12.2f} {(1 - stack_rmse_p / tso_rmse_p) * 100:>+11.1f}%')
print(f'{"MAPE (%)":<14} {stack_mape_p:>12.2f}')

Metric            Stacking          TSO  Improvement
----------------------------------------------------
MAE (EUR/MWh)       4.58         8.87       +48.4%
RMSE (EUR/MWh)      6.20        11.90       +47.9%
MAPE (%)            8.81


In [10]:
# Export stacking_price.json
sample_data_p = []
for w in range(9, 16):
    if w >= len(test_window_starts):
        break
    ws = test_window_starts[w]
    for h in range(prediction_length):
        t = df['time'].iloc[ws + h]
        sample_data_p.append({
            'time': t.strftime('%Y-%m-%d %H:%M'),
            'actual': round(float(price_actuals[w, h]), 2),
            'predicted': round(float(price_preds[w, h]), 2),
            'tso': round(float(price_tso[w, h]), 2),
        })

output_p = {
    'target': 'price',
    'model': 'Stacking Ensemble (XGB + LGBM + Ridge + RF)',
    'prediction_length_hours': prediction_length,
    'context_length_hours': context_length,
    'metrics': {
        'mae': round(float(stack_mae_p), 2),
        'rmse': round(float(stack_rmse_p), 2),
        'mape': round(float(stack_mape_p), 2),
        'tso_mae': round(float(tso_mae_p), 2),
        'tso_rmse': round(float(tso_rmse_p), 2),
    },
    'sample_forecast': sample_data_p,
}

with open('../dashboard/public/data/stacking_price.json', 'w') as f:
    json.dump(output_p, f, indent=2)

print(f'Saved stacking_price.json (MAE: {output_p["metrics"]["mae"]} EUR/MWh)')

Saved stacking_price.json (MAE: 4.58 EUR/MWh)
